# Setup

In [1]:
from Bio import Entrez
import pandas as pd
import json # for pretty printing

In [3]:
Entrez.email = "giorgiocoalshopping1@gmail.com"

# The Nucleotide Database Sequencies

In [4]:
handle = Entrez.esearch(db = "nucleotide", 
                        term = '"Toxoplasma gondii"[Organism]', 
                        RetMax = 300000,
                        retmode = "xml")
search_record = Entrez.read(handle)

In [5]:
print(search_record.keys())
print(search_record["QueryTranslation"])
print(len(search_record["IdList"]))

dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'TranslationStack', 'QueryTranslation'])
"Toxoplasma gondii"[Organism]
227607


In [24]:
record = {}
counter = 0
NUM_ARTICLES = 9999
ret_start = 0
lenght_batch = 1
for counter in range(0,19):
    print(f"Loading batch:{counter}")
    handle = Entrez.efetch(db = "nucleotide", retmax = NUM_ARTICLES, retstart = ret_start, 
                            id = ','.join(search_record["IdList"]), retmode = "xml")
    record[counter] = Entrez.read(handle)
    ret_start += NUM_ARTICLES
    handle.close()

Loading batch:0
Loading batch:1
Loading batch:2
Loading batch:3
Loading batch:4
Loading batch:5
Loading batch:6
Loading batch:7
Loading batch:8
Loading batch:9
Loading batch:10
Loading batch:11
Loading batch:12
Loading batch:13
Loading batch:14
Loading batch:15
Loading batch:16
Loading batch:17
Loading batch:18


In [29]:
print(type(record))
print(type(record[0]))
print(record[0][0].keys())
print(record[0][0])
print(json.dumps(record[0][100], indent = 2))
# for i in record[0]:
#     print(json.dumps(i, indent = 2))

<class 'dict'>
<class 'Bio.Entrez.Parser.ListElement'>
dict_keys(['GBSeq_locus', 'GBSeq_length', 'GBSeq_strandedness', 'GBSeq_moltype', 'GBSeq_topology', 'GBSeq_division', 'GBSeq_update-date', 'GBSeq_create-date', 'GBSeq_definition', 'GBSeq_primary-accession', 'GBSeq_accession-version', 'GBSeq_other-seqids', 'GBSeq_source', 'GBSeq_organism', 'GBSeq_taxonomy', 'GBSeq_references', 'GBSeq_comment', 'GBSeq_feature-table', 'GBSeq_sequence'])
{'GBSeq_locus': 'OQ402957', 'GBSeq_length': '597', 'GBSeq_strandedness': 'double', 'GBSeq_moltype': 'DNA', 'GBSeq_topology': 'linear', 'GBSeq_division': 'INV', 'GBSeq_update-date': '26-FEB-2023', 'GBSeq_create-date': '26-FEB-2023', 'GBSeq_definition': 'Toxoplasma gondii voucher W22_6329 Apico allele II marker genomic sequence; apicoplast', 'GBSeq_primary-accession': 'OQ402957', 'GBSeq_accession-version': 'OQ402957.1', 'GBSeq_other-seqids': ['gb|OQ402957.1|', 'gi|2447291127'], 'GBSeq_source': 'apicoplast Toxoplasma gondii', 'GBSeq_organism': 'Toxoplasma 

In [26]:
full_record = []
for i in record:
    full_record += record[i]

Totale 

In [27]:
len(full_record)

189981

In [28]:
full_record[1]['GBSeq_locus']

'OQ402956'

## Parsing

In [142]:
id_article = {}
for i, records in enumerate(full_record):
    # Create empty dictionary for each given article id
    id_article[i] = {}
    # Title
    try:
        id_article[i]['GBSeq_locus'] = records['GBSeq_locus']
    except:
        id_article[i]['GBSeq_locus'] = None
    # Keywordlist
    try:
        id_article[i]['GBSeq_length'] = records['GBSeq_length']
    except:
        id_article[i]['GBSeq_length'] = None
    # Journal Title
    try:
        id_article[i]['GBSeq_strandedness'] = records['GBSeq_strandedness']
    except:
        id_article[i]['GBSeq_strandedness'] = None
    # Journal ISO
    try:
        id_article[i]['GBSeq_create-date'] = records['GBSeq_create-date']
    except:
        id_article[i]['GBSeq_create-date'] = None
    # Journal Country
    try:
        id_article[i]['GBSeq_update-date'] = records['GBSeq_update-date']
    except:
        id_article[i]['GBSeq_update-date'] = None
    try:
        id_article[i]['GBSeq_definition'] = records['GBSeq_definition']
    except:
        id_article[i]['GBSeq_definition'] = None
    # Date (submission?) as MM/DD/YYYY
    try: 
        id_article[i]['Submission Type'] = records['GBSeq_references'][1]['GBReference_title']
    except:
        id_article[i]['Submission Type'] = None
    # Completed date as MM/DD/YYYY
    try: 
        id_article[i]['Sequencing Type'] = records['GBSeq_comment']
    except:
        id_article[i]['Sequencing Type'] = None
    # Revised date as MM/DD/YYYY
    root = records['GBSeq_feature-table'][0]['GBFeature_quals']
    try:
        id_article[i]['mol_type'] = [dictemp['GBQualifier_value'] for dictemp in root if dictemp['GBQualifier_name'] == 'mol_type'][0]
    except:
        id_article[i]['mol_type'] = None
    # Publication year
    try:
        id_article[i]['isolation_source'] = [dictemp['GBQualifier_value'] for dictemp in root if dictemp['GBQualifier_name'] == 'isolation_source'][0]
    except:
        id_article[i]['isolation_source'] = None
    try:
        id_article[i]['host'] = [dictemp['GBQualifier_value'] for dictemp in root if dictemp['GBQualifier_name'] == 'host'][0]
    except:
        id_article[i]['host'] = None
    try:
        id_article[i]['country'] = [dictemp['GBQualifier_value'] for dictemp in root if dictemp['GBQualifier_name'] == 'country'][0]
    except:
        id_article[i]['country'] = None
    try:
        id_article[i]['collection_date'] = [dictemp['GBQualifier_value'] for dictemp in root if dictemp['GBQualifier_name'] == 'collection_date'][0]
    except:
        id_article[i]['collection_date'] = None

In [169]:
import random
print(json.dumps(random.choice(list(id_article.values())), indent = 2))

{
  "GBSeq_locus": "AHZP02000810",
  "GBSeq_length": "19924",
  "GBSeq_strandedness": "double",
  "GBSeq_create-date": "23-MAR-2016",
  "GBSeq_update-date": "23-MAR-2016",
  "GBSeq_definition": "Toxoplasma gondii TgCatPRC2 asmbl.931, whole genome shotgun sequence",
  "Submission Type": "Direct Submission",
  "Sequencing Type": "This work was supported by the National Institute of Allergy and Infectious Diseases (NIAID), Genome Sequencing Centers for Infectious Diseases (GSCID) program. This sequence record is co-owned by JCVI and EuPathDB. Additional Toxoplasma gondii resources are available at ToxoDB, http://toxodb.org/toxo/ and at JCVI, http://gsc.jcvi.org/projects/gsc/t_gondii/index.shtml.~~The VEuPathDB Bioinformatics Resource Center (www.veupathdb.org) is responsible for the assembled genome sequence and its annotation as of March 2022. Annotation and related data can be viewed and updated at http://veupathdb.org and inquiries can be sent to help@veupathdb.org.; ##Genome-Assembly-

Create the dataframe

In [170]:
id_article_df = pd.DataFrame.from_dict(id_article, orient = "index")
id_article_df.head()

,GBSeq_locus,GBSeq_length,GBSeq_strandedness,GBSeq_create-date,GBSeq_update-date,GBSeq_definition,Submission Type,Sequencing Type,mol_type,isolation_source,host,country,collection_date
0,OQ402957,597,double,26-FEB-2023,26-FEB-2023,Toxoplasma gondii voucher W22_6329 Apico allel...,Direct Submission,##Assembly-Data-START## ; Sequencing Technolog...,genomic DNA,brain,Castor fiber,Switzerland,2022
1,OQ402956,597,double,26-FEB-2023,26-FEB-2023,Toxoplasma gondii voucher W22_6002 Apico allel...,Direct Submission,##Assembly-Data-START## ; Sequencing Technolog...,genomic DNA,brain,Castor fiber,Switzerland,2022
2,OQ402955,597,double,26-FEB-2023,26-FEB-2023,Toxoplasma gondii voucher W21_2282 Apico allel...,Direct Submission,##Assembly-Data-START## ; Sequencing Technolog...,genomic DNA,brain,Castor fiber,Switzerland,2021
3,OQ402954,597,double,26-FEB-2023,26-FEB-2023,Toxoplasma gondii voucher W21_1498 Apico allel...,Direct Submission,##Assembly-Data-START## ; Sequencing Technolog...,genomic DNA,brain,Castor fiber,Switzerland,2021
4,OQ402953,597,double,26-FEB-2023,26-FEB-2023,Toxoplasma gondii voucher W21_1294 Apico allel...,Direct Submission,##Assembly-Data-START## ; Sequencing Technolog...,genomic DNA,brain,Castor fiber,Switzerland,2021


Save the dataset

In [171]:
id_article_df.to_csv("Data/toxoplasma_gondii_nucleotide.csv", encoding = 'utf-8')

# PubMed Articles IDs Search and Retrieval

## Batch Articles IDs Search per Data Range

For PubMed, ESearch can only retrieve the first 10,000 records matching the query. To obtain more than 10,000 PubMed records, consider using <EDirect> that contains additional logic to batch PubMed search results automatically so that an arbitrary number can be retrieved.

To avoid the maximum number of articles retrieved limitation, i decided to search them in batches of maximum 9999 articles, using a date range as limiter.

Fetching the most recent 9934 articles published from **2009** to **2023**:

In [3]:
NUM_ARTICLES = 10000
search_record = {}
handle = Entrez.esearch(db = "pubmed", 
                        retmax = NUM_ARTICLES, 
                        retstart = 0, 
                        term = '(("toxoplasma gondii"[Title/Abstract]) AND ("2009/01/01"[Date - Publication] : "2023/03/22"[Date - Publication]))', 
                        idtype = "acc",
                        retmode = "xml")
search_record[">2009"] = Entrez.read(handle)
handle.close()

In [4]:
print(search_record[">2009"].keys())
print(search_record[">2009"]["QueryTranslation"])
print(len(search_record[">2009"]["IdList"]))

dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'QueryTranslation', 'WarningList'])
"toxoplasma gondii"[Title/Abstract] AND 2009/01/01:2023/03/22[Date - Publication]
9947


Fetching older 7770 articles published **before 2009**:

In [ ]:
NUM_ARTICLES = 10000
handle = Entrez.esearch(db = "pubmed", 
                        retmax = NUM_ARTICLES, 
                        retstart = 0, 
                        term = '(("toxoplasma gondii"[Title/Abstract]) AND ("1900/01/01"[Date - Publication] : "2008/12/31"[Date - Publication]))', 
                        idtype = "acc",
                        retmode = "xml")
search_record["<2009"] = Entrez.read(handle)
handle.close()

In [ ]:
print(search_record["<2009"].keys())
print(search_record["<2009"]["WarningList"])
print(len(search_record["<2009"]["IdList"]))

dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'QueryTranslation', 'WarningList'])
{'QuotedPhraseNotFound': [], 'PhraseIgnored': [], 'OutputMessage': ['Restrictions achieved. start and count adjusted to 0, 9999']}
7770


## Merging the batches and Remove Duplicates

Now I'll merge the IDs Batches obtaining a single list of 10000+ IDs.

In [ ]:
search_record_full = list(search_record[">2009"]["IdList"]) + list(search_record["<2009"]["IdList"])

Number of articles to fetch

In [ ]:
# Number of articles
print(len(search_record_full))
# Number of unique articles
print(len(set(search_record_full)))

17717
17651


Removing the duplicates

In [ ]:
search_record_full = list(set(search_record_full))

In [ ]:
len(search_record_full)

17651

# Fetch Bibliographic Information from retrieved IDs

Also `Entrez.efetch` has a maximum number of 'IDs' which can be retrieved, it is important to slice the list of IDs during fetching.

Fetch detailed information about every article in the list

In [ ]:
# Retrieve only `top_n` most recent articles
records = {}
half = len(search_record_full) // 2
handle = Entrez.efetch(db = "pubmed", 
                       # Convert list to string with comma as separator 
                       # to pass to Entrez.efetch
                       id = ','.join(search_record_full[:half]),
                       retmode = "xml")
records['1stHalf'] = Entrez.read(handle)
handle.close()
handle = Entrez.efetch(db = "pubmed", 
                       # Convert list to string with comma as separator 
                       # to pass to Entrez.efetch
                       id = ','.join(search_record_full[half:]),
                       retmode = "xml")
records['2ndHalf'] = Entrez.read(handle)
handle.close()

Record dictionary details:

In [ ]:
# First Half
print(records['1stHalf'].keys())

print(len(records['1stHalf']["PubmedArticle"]))
print(type(records['1stHalf']["PubmedArticle"]))
print(type(records['1stHalf']["PubmedArticle"][1]))

# Second Half
print(len(records['2ndHalf']["PubmedArticle"]))
print(type(records['2ndHalf']["PubmedArticle"]))
print(type(records['2ndHalf']["PubmedArticle"][1]))

dict_keys(['PubmedBookArticle', 'PubmedArticle'])
8820
<class 'list'>
<class 'Bio.Entrez.Parser.DictionaryElement'>
8822
<class 'list'>
<class 'Bio.Entrez.Parser.DictionaryElement'>


Merging the two records dictionaries in a single one:

In [ ]:
full_records = {}
full_records['PubmedArticle'] = records['1stHalf']["PubmedArticle"] + records['2ndHalf']["PubmedArticle"]
full_records['PubmedBookArticle'] = records['1stHalf']["PubmedBookArticle"] + records['2ndHalf']["PubmedBookArticle"]

In [ ]:
print(len(full_records["PubmedArticle"]))
print(type(full_records["PubmedArticle"]))
print(type(full_records["PubmedArticle"][1]))

17642
<class 'list'>
<class 'Bio.Entrez.Parser.DictionaryElement'>


In [ ]:
print(json.dumps(full_records['PubmedArticle'][100], indent = 2))

{
  "MedlineCitation": {
    "CitationSubset": [
      "IM"
    ],
    "OtherID": [],
    "GeneralNote": [],
    "SpaceFlightMission": [],
    "OtherAbstract": [],
    "KeywordList": [
      [
        "Artemether",
        "Cognition",
        "Psychopathology",
        "Schizophrenia",
        "Toxoplasma gondii"
      ]
    ],
    "PMID": "24656425",
    "DateCompleted": {
      "Year": "2014",
      "Month": "11",
      "Day": "24"
    },
    "DateRevised": {
      "Year": "2022",
      "Month": "03",
      "Day": "16"
    },
    "Article": {
      "Language": [
        "eng"
      ],
      "ELocationID": [
        "10.1016/j.jpsychires.2014.02.016",
        "S0022-3956(14)00056-9"
      ],
      "ArticleDate": [
        {
          "Year": "2014",
          "Month": "02",
          "Day": "28"
        }
      ],
      "Journal": {
        "ISSN": "1879-1379",
        "JournalIssue": {
          "Volume": "53",
          "PubDate": {
            "Year": "2014",
            "Month": 

# Parsing retrieved articles and build a dataframe

## JSON Parsing

We create multiple dataframes containing information about:
-  Retrieved Article
-  ~~Journal that published the articles~~

First of all we'll extract some basic information about the retrieved articles:
- PubMed ID
- Title
- Abstract Text
- Dates
  - Completed Date
  - Revised Date
  - Published Year
- Language
- Authors List
- PublicationTypeList
- Journal Country

In [ ]:
# Create empty dictionary to store pubmed_id its info
id_article = {}
counter = 0
for record in full_records['PubmedArticle']:
    #print(counter)
    counter += 1
    # Root element to access all data about a given article
    root = record['MedlineCitation']['Article']
    # Get pubmed id to be used for dictionary key access
    uid = record['MedlineCitation']['PMID']
    #print(uid)
    # Create empty dictionary for each given article id
    id_article[uid] = {}
    # Title
    try:
        id_article[uid]['Title'] = root['ArticleTitle']
    except:
        id_article[uid]['Title'] = None
    # Keywordlist
    try:
        id_article[uid]['Keyword List'] = " | ".join(record['MedlineCitation']['KeywordList'][0])
    except:
        id_article[uid]['Keyword List'] = None
    # Journal Title
    try:
        id_article[uid]['Journal Name'] = record['MedlineCitation']['Article']['Journal']['Title']
    except:
        id_article[uid]['Journal Name'] = None
    # Journal ISO
    try:
        id_article[uid]['Journal ISO'] = record['MedlineCitation']['MedlineJournalInfo']['MedlineTA']
    except:
        id_article[uid]['Journal ISO'] = None
    # Journal Country
    try:
        id_article[uid]['Journal Country'] = record['MedlineCitation']['MedlineJournalInfo']['Country']
    except:
        id_article[uid]['Journal Country'] = None
    # Abstract text different sections like results, conclusions, 
    # that are in separate elements of the list are joined together
    try:
        id_article[uid]['Abstract'] = "\t".join(root['Abstract']['AbstractText']) if 'Abstract' in root.keys() else None
    except:
        id_article[uid]['Abstract'] = None
    # Date (submission?) as MM/DD/YYYY
    try: 
        id_article[uid]['ArticleDate'] = "/".join(root['ArticleDate'][0].values()) if len(root['ArticleDate']) != 0 else None 
    except:
        id_article[uid]['ArticleDate'] = None
    # Completed date as MM/DD/YYYY
    try: 
        id_article[uid]['CompletedDate'] = "/".join(record['MedlineCitation']['DateCompleted'].values()) if len(record['MedlineCitation']['DateCompleted']) != 0 else None
    except:
        id_article[uid]['CompletedDate'] = None
    # Revised date as MM/DD/YYYY
    try:
        id_article[uid]['RevisedDate'] = "/".join(record['MedlineCitation']['DateRevised'].values()) if len(record['MedlineCitation']['DateRevised']) != 0 else None
    except:
        id_article[uid]['RevisedDate'] = None
    # Publication year
    try:
        id_article[uid]['PublicationYear'] = record['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']["Year"]
    except:
        id_article[uid]['PublicationYear'] = None
    # Language(s)
    try:
        id_article[uid]['Language'] = " | ".join(root['Language'])
    except:
        id_article[uid]['Language'] = None
    # Authors list
    try:
        id_article[uid]["AuthorList"] = " | ".join([i['LastName'] + ", " + i['Initials']\
                                                for i in root['AuthorList'] if 'LastName' and 'Initials' in i.keys()]) if 'AuthorList' in root.keys() else None
    except:
        id_article[uid]["AuthorList"] = None
    # List of publication types
    try: 
        id_article[uid]['PublicationTypeList'] = " | ".join(root['PublicationTypeList'])
    except:
        id_article[uid]['PublicationTypeList'] = None
        

Example of the parsed data:

In [ ]:
type(id_article)

dict

In [ ]:
import random
print(json.dumps(random.choice(list(id_article.values())), indent = 2))

{
  "Title": "[Acute infection of Toxoplasma gondii affects the level of oxygen free radicals in serum and testes of mice].",
  "Keyword List": null,
  "Journal Name": "Zhongguo ji sheng chong xue yu ji sheng chong bing za zhi = Chinese journal of parasitology & parasitic diseases",
  "Journal ISO": "Zhongguo Ji Sheng Chong Xue Yu Ji Sheng Chong Bing Za Zhi",
  "Journal Country": "China",
  "Abstract": "To observe the impact of Toxoplasma gondii infection on the level of oxygen free radicals and antioxidant enzyme in serum and testes of mice.\t40 BALB/c male mice were randomly divided into four groups. Five mice from each group were injected intraperitoneally with 2.5 x 10(3) tachyzoites of T. gondii, the others received PBS. Mice were sacrificed on the 1st, 3rd, 5th, and 7th day after inoculation. Samples of serum and testes were collected to determine the content of oxygen free radicals and superoxide dismutase (SOD).\tThe concentration of the oxygen free radicals (NO, *OH, O2-) in s

Create the dataframe

In [ ]:
id_article_df = pd.DataFrame.from_dict(id_article, orient = "index")
id_article_df.head()

,Title,Keyword List,Journal Name,Journal ISO,Journal Country,Abstract,ArticleDate,CompletedDate,RevisedDate,PublicationYear,Language,AuthorList,PublicationTypeList
8579614,Purification and cloning of a soluble ATP-diph...,None,Biochemical and biophysical research communica...,Biochem Biophys Res Commun,United States,"A soluble ATP-diphosphohydrolase (apyrase, EC ...",None,1996/03/12,2007/11/14,1996,eng,"Handa, M | Guidotti, G",Comparative Study | Journal Article | Research...
769615,Mechanism of uptake of strontium isotopes in a...,None,The American review of respiratory disease,Am Rev Respir Dis,United States,Observations on experimental aspergillosis of ...,None,1976/06/02,2014/11/20,1976,eng,"Rawal, BD | Adiseshan, N",Journal Article
19147421,[Multicenter study on the prevention of congen...,None,Medicina,Medicina (B Aires),Argentina,Toxoplasmosis is an infection caused by Toxopl...,None,2009/11/16,2009/01/16,2008,spa,"Carral, L | Kaufer, F | Durlach, R | Freuler, ...",English Abstract | Journal Article | Multicent...
32820712,Modelling <i>Toxoplasma gondii</i> infection i...,Cerebral organoid | Toxoplasma gondii | diseas...,Emerging microbes & infections,Emerg Microbes Infect,United States,Pluripotent stem cell-derived cerebral organoi...,None,2021/04/21,2021/04/28,2020,eng,"Seo, HH | Han, HW | Lee, SE | Hong, SH | Cho, ...",Journal Article
30891398,Evidence of red panda as an intermediate host ...,China | Genotype | Isolation | Red panda (Ailu...,International journal for parasitology. Parasi...,Int J Parasitol Parasites Wildl,England,<i>Toxoplasma gondii</i> has been found to inf...,2019/02/23,None,2020/10/01,2019,eng,"Yang, Y | Dong, H | Su, R | Li, T | Jiang, N |...",Journal Article


## Abstract Text Cleaning

### Old

Remove `<i>` and `</i>` tags from Title and Abstract

In [144]:
def remove_tags(text):
    return text.replace('<i>', '').replace('</i>', '')

id_article_df['Title'] = id_article_df['Title'].map(remove_tags)
id_article_df['PreprocessedAbstract'] = id_article_df['PreprocessedAbstract'].map(remove_tags)

Remove Puntuaction

In [145]:
def remove_punctuation(text):
    return re.sub('[^\w\s]', '', text)
id_article_df['PreprocessedAbstract'] = id_article_df['PreprocessedAbstract'].map(remove_punctuation)

Lower the text

In [146]:
def to_lower(text):
    return text.lower()
id_article_df['PreprocessedAbstract'] = id_article_df['PreprocessedAbstract'].map(to_lower)

### Functions

Import

In [7]:
id_article_df=pd.read_csv("./Data/toxoplasma_gondii_pubmed.csv", index_col = "pubmed_id")

In [8]:
id_article_df.head()

,Title,Keyword List,Journal Name,Journal ISO,Journal Country,Abstract,ArticleDate,CompletedDate,RevisedDate,PublicationYear,Language,AuthorList,PublicationTypeList,PreprocessedAbstract
pubmed_id,,,,,,,,,,,,,,
8579614,Purification and cloning of a soluble ATP-diph...,[''],Biochemical and biophysical research communica...,Biochem Biophys Res Commun,United States,"A soluble ATP-diphosphohydrolase (apyrase, EC ...",NaN,1996/03/12,2007/11/14,1996.0,eng,"['Handa, M ', ' Guidotti, G']","['Comparative Study ', ' Journal Article ', "" ...",a soluble atpdiphosphohydrolase apyrase ec 361...
769615,Mechanism of uptake of strontium isotopes in a...,[''],The American review of respiratory disease,Am Rev Respir Dis,United States,Observations on experimental aspergillosis of ...,NaN,1976/06/02,2014/11/20,1976.0,eng,"['Rawal, BD ', ' Adiseshan, N']",['Journal Article'],observations on experimental aspergillosis of ...
19147421,Multicenter study on the prevention of congeni...,[''],Medicina,Medicina (B Aires),Argentina,Toxoplasmosis is an infection caused by Toxopl...,NaN,2009/11/16,2009/01/16,2008.0,spa,"['Carral, L ', ' Kaufer, F ', ' Durlach, R ', ...","['English Abstract ', ' Journal Article ', ' M...",toxoplasmosis is an infection caused by toxopl...
32820712,Modelling Toxoplasma gondii infection in human...,"['Cerebral organoid ', ' Toxoplasma gondii ', ...",Emerging microbes & infections,Emerg Microbes Infect,United States,Pluripotent stem cell-derived cerebral organoi...,NaN,2021/04/21,2021/04/28,2020.0,eng,"['Seo, HH ', ' Han, HW ', ' Lee, SE ', ' Hong,...",['Journal Article'],pluripotent stem cellderived cerebral organoid...
30891398,Evidence of red panda as an intermediate host ...,"['China ', ' Genotype ', ' Isolation ', ' Red ...",International journal for parasitology. Parasi...,Int J Parasitol Parasites Wildl,England,<i>Toxoplasma gondii</i> has been found to inf...,2019/02/23,NaN,2020/10/01,2019.0,eng,"['Yang, Y ', ' Dong, H ', ' Su, R ', ' Li, T '...",['Journal Article'],toxoplasma gondii has been found to infect alm...


In [9]:
import re 
import os
import json
import pandas as pd
#import pprint
import seaborn as sns
import numpy as np 
from collections import Counter 
from matplotlib import pyplot as plt
#import statistics
import gc
#from IPython.display import clear_output

import multiprocessing
#from functools import partial
from multiprocessing import  Pool

#from pandarallel import pandarallel

import swifter
# from swifter import set_defaults
# cores = multiprocessing.cpu_count()
# num_process = cores
# set_defaults(
#     npartitions=cores,
#     dask_threshold=1,
#     scheduler="processes",
#     progress_bar=True,
#     progress_bar_desc=None,
#     allow_dask_on_strings=False,
#     force_parallel=False,
# )

from tqdm import tqdm, tqdm_pandas

# nlp
from num2words import num2words

#import string

#from bs4 import BeautifulSoup 
#import time

from textblob import TextBlob

#from pygments import highlight
#from pygments.lexers import get_lexer_by_name
#from pygments.formatters import HtmlFormatter
#from pygments.lexers import guess_lexer

#import emoji
#import demoji

import re
import contractions
#from contractions import contractions_dict

import nltk
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
#nltk.download('wordnet')
from nltk.corpus import stopwords
#import gensim
#from gensim.utils import simple_preprocess
#import gensim.corpora as corpora

# nltk.download('averaged_perceptron_tagger')
# nltk.download('tagsets')
# nltk.download('universal_tagset')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Functions

In [10]:
def remove_tags(text):
    return text.replace('<i>', '').replace('</i>', '')

def remove_html_tags(sentence):
    pattern = re.compile("<.*?>") 
    cleaned_sentence = re.sub(pattern,'',sentence).strip()
    return cleaned_sentence

def remove_html_entities(sentence):
    pattern = re.compile("&[a-z0-9]+|&#[0-9]{1,6}|&#x[0-9a-f]{1,6}")
    cleaned_sentence = re.sub(pattern,'',sentence).strip()
    return cleaned_sentence

def remove_extra_whitespaces(text):
    cleaned_sentence = re.sub(r'^\s*|\s\s*', ' ', text).strip()
    return cleaned_sentence

def remove_urls(sentence):
    http_pattern = re.compile(r"http\S+") 
    cleaned_sentence = re.sub(http_pattern,'',sentence).strip()
    www_pattern = re.compile(r"www\S+") 
    cleaned_sentence = re.sub(www_pattern,'',cleaned_sentence)
    return cleaned_sentence

def replace_ordinal_numbers(text):
    re_results = re.findall('(\d+(st|nd|rd|th))', text)
    for enitre_result, suffix in re_results:
        num = int(enitre_result[:-len(suffix)])
        text = text.replace(enitre_result, "")
    return text

def remove_numbers(phrase):
  cleaned_sentence=re.sub(r'\d+', '',phrase)
  return cleaned_sentence

def process_numbers(phrase):
    phrase = replace_ordinal_numbers(phrase)
    phrase = remove_numbers(phrase)
    return phrase

def remove_control_chars(phrase):
    cleaned_sentence = re.sub(r'(\\ u |\\ n |\\ v |\\ m | \\ )', ' ',phrase)
    return cleaned_sentence

# WORD PROCESSING

def to_lower(phrase):
  phrase = phrase.lower()
  return phrase

def fix_and_expand_eng_contradictions(phrase):
    phrase = contractions.fix(phrase, slang=True)
    return phrase

def remove_special_characters_punctuations(sentence):
    cleaned_text  = re.sub(r"[^a-zA-Z]+",' ',sentence).strip()
    return cleaned_text

# WORD ANALYSIS

def stop_words_1char_removal(text, stopwords_list = stopwords.words('english')):
  return ([word for word in text if (word not in stopwords_list) and (len(word) > 1)])

def lemmaSentence(token_words):
    lemma_text=[]
    for word in token_words:
        lemma_text.append(WordNetLemmatizer().lemmatize(word))
    return lemma_text

def text_Tokenization(sentences):
  # Tokenization
  return list(map(word_tokenize, sentences))

def text_stop_words_1char_removal(sentences):
  # Stopwords removal
  return list(map(stop_words_1char_removal, sentences))

def text_lemmatizer(senteces):
  return list(map(lemmaSentence, senteces))

def text_normalization_full(sentences):
  # Text Cleaning
  try:
    sentences = remove_tags(sentences)
    sentences = remove_html_tags(sentences)
    sentences = remove_html_entities(sentences)
    sentences = remove_extra_whitespaces(sentences)
    sentences = remove_urls(sentences)
    sentences = process_numbers(sentences)
    sentences = remove_control_chars(sentences)
    sentences = remove_extra_whitespaces(sentences)
    # Word Processing
    sentences = to_lower(sentences)
    sentences = fix_and_expand_eng_contradictions(sentences)
    sentences = remove_special_characters_punctuations(sentences)
    # Normalizing
    sentences = word_tokenize(sentences)
    sentences = stop_words_1char_removal(sentences)
    sentences = lemmaSentence(sentences)
  except:
    print(sentences)
  return sentences

In [11]:
# Copy abstract to new column
id_article_df['PreprocessedAbstract'] = id_article_df['Abstract']

Convert all None elements to empty strings (to be able to use re functions)

In [13]:
id_article_df['PreprocessedAbstract'] = id_article_df['PreprocessedAbstract'].fillna('')

Normalizing in multiprocessing

In [14]:
id_article_df["PreprocessedAbstract"] = id_article_df["PreprocessedAbstract"].swifter.apply(text_normalization_full)

Pandas Apply:   0%|          | 0/17642 [00:00<?, ?it/s]

In [20]:
id_article_df.sample(n=1, axis=0)['PreprocessedAbstract']

pubmed_id
26293079    [toxoplasma, gondii, endemic, pathogen, approx...
Name: PreprocessedAbstract, dtype: object

Remove squared brackets from Title

In [ ]:
def remove_squared_brackets(text):
    return text.replace('[', '').replace(']', '')

id_article_df['Title'] = id_article_df['Title'].map(remove_squared_brackets)

In [27]:
id_article_df[id_article_df['Keyword List'] == "['']"]['Keyword List'] = None

C:\Users\giorg\AppData\Local\Temp\ipykernel_34104\4119480717.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_article_df[id_article_df['Keyword List'] == "['']"]['Keyword List'] = None


In [30]:
id_article_df.head()

,Title,Keyword List,Journal Name,Journal ISO,Journal Country,Abstract,ArticleDate,CompletedDate,RevisedDate,PublicationYear,Language,AuthorList,PublicationTypeList,PreprocessedAbstract
pubmed_id,,,,,,,,,,,,,,
8579614,Purification and cloning of a soluble ATP-diph...,[''],Biochemical and biophysical research communica...,Biochem Biophys Res Commun,United States,"A soluble ATP-diphosphohydrolase (apyrase, EC ...",NaN,1996/03/12,2007/11/14,1996.0,eng,"['Handa, M ', ' Guidotti, G']","['Comparative Study ', ' Journal Article ', "" ...","[soluble, atp, diphosphohydrolase, apyrase, ec..."
769615,Mechanism of uptake of strontium isotopes in a...,[''],The American review of respiratory disease,Am Rev Respir Dis,United States,Observations on experimental aspergillosis of ...,NaN,1976/06/02,2014/11/20,1976.0,eng,"['Rawal, BD ', ' Adiseshan, N']",['Journal Article'],"[observation, experimental, aspergillosis, cho..."
19147421,Multicenter study on the prevention of congeni...,[''],Medicina,Medicina (B Aires),Argentina,Toxoplasmosis is an infection caused by Toxopl...,NaN,2009/11/16,2009/01/16,2008.0,spa,"['Carral, L ', ' Kaufer, F ', ' Durlach, R ', ...","['English Abstract ', ' Journal Article ', ' M...","[toxoplasmosis, infection, caused, toxoplasma,..."
32820712,Modelling Toxoplasma gondii infection in human...,"['Cerebral organoid ', ' Toxoplasma gondii ', ...",Emerging microbes & infections,Emerg Microbes Infect,United States,Pluripotent stem cell-derived cerebral organoi...,NaN,2021/04/21,2021/04/28,2020.0,eng,"['Seo, HH ', ' Han, HW ', ' Lee, SE ', ' Hong,...",['Journal Article'],"[pluripotent, stem, cell, derived, cerebral, o..."
30891398,Evidence of red panda as an intermediate host ...,"['China ', ' Genotype ', ' Isolation ', ' Red ...",International journal for parasitology. Parasi...,Int J Parasitol Parasites Wildl,England,<i>Toxoplasma gondii</i> has been found to inf...,2019/02/23,NaN,2020/10/01,2019.0,eng,"['Yang, Y ', ' Dong, H ', ' Su, R ', ' Li, T '...",['Journal Article'],"[toxoplasma, gondii, found, infect, almost, wa..."


## Convert Authors, PublicationTypeList and Keywords List in lists

In [ ]:
# Definizione della funzione per trasformare una stringa in una lista di stringhe utilizzando il separatore '|'
def string_to_list(text):
    return text.split('|')

# Applicazione della funzione alla colonna 'my_column' tramite apply()
id_article_df['AuthorList'] = id_article_df['AuthorList'].fillna('')
id_article_df['AuthorList'] = id_article_df['AuthorList'].apply(string_to_list)

id_article_df['PublicationTypeList'] = id_article_df['PublicationTypeList'].fillna('')
id_article_df['PublicationTypeList'] = id_article_df['PublicationTypeList'].apply(string_to_list)

id_article_df['Keyword List'] = id_article_df['Keyword List'].fillna('')
id_article_df['Keyword List'] = id_article_df['Keyword List'].apply(string_to_list)

In [ ]:
id_article_df.head()

,Title,Keyword List,Journal Name,Journal ISO,Journal Country,Abstract,ArticleDate,CompletedDate,RevisedDate,PublicationYear,Language,AuthorList,PublicationTypeList,PreprocessedAbstract
8579614,Purification and cloning of a soluble ATP-diph...,[],Biochemical and biophysical research communica...,Biochem Biophys Res Commun,United States,"A soluble ATP-diphosphohydrolase (apyrase, EC ...",None,1996/03/12,2007/11/14,1996,eng,"[Handa, M , Guidotti, G]","[Comparative Study , Journal Article , Resea...",a soluble atpdiphosphohydrolase apyrase ec 361...
769615,Mechanism of uptake of strontium isotopes in a...,[],The American review of respiratory disease,Am Rev Respir Dis,United States,Observations on experimental aspergillosis of ...,None,1976/06/02,2014/11/20,1976,eng,"[Rawal, BD , Adiseshan, N]",[Journal Article],observations on experimental aspergillosis of ...
19147421,Multicenter study on the prevention of congeni...,[],Medicina,Medicina (B Aires),Argentina,Toxoplasmosis is an infection caused by Toxopl...,None,2009/11/16,2009/01/16,2008,spa,"[Carral, L , Kaufer, F , Durlach, R , Freul...","[English Abstract , Journal Article , Multic...",toxoplasmosis is an infection caused by toxopl...
32820712,Modelling Toxoplasma gondii infection in human...,"[Cerebral organoid , Toxoplasma gondii , dis...",Emerging microbes & infections,Emerg Microbes Infect,United States,Pluripotent stem cell-derived cerebral organoi...,None,2021/04/21,2021/04/28,2020,eng,"[Seo, HH , Han, HW , Lee, SE , Hong, SH , ...",[Journal Article],pluripotent stem cellderived cerebral organoid...
30891398,Evidence of red panda as an intermediate host ...,"[China , Genotype , Isolation , Red panda (...",International journal for parasitology. Parasi...,Int J Parasitol Parasites Wildl,England,<i>Toxoplasma gondii</i> has been found to inf...,2019/02/23,None,2020/10/01,2019,eng,"[Yang, Y , Dong, H , Su, R , Li, T , Jiang...",[Journal Article],toxoplasma gondii has been found to infect alm...


## Save the dataset

In [31]:
id_article_df.to_csv("Data/toxoplasma_gondii_pubmed.csv", index_label = "pubmed_id", encoding = 'utf-8')

In [32]:
print(id_article_df.columns)

Index(['Title', 'Keyword List', 'Journal Name', 'Journal ISO',
       'Journal Country', 'Abstract', 'ArticleDate', 'CompletedDate',
       'RevisedDate', 'PublicationYear', 'Language', 'AuthorList',
       'PublicationTypeList', 'PreprocessedAbstract'],
      dtype='object')
